In [51]:
%%writefile phonebook_search.cu
#include <bits/stdc++.h>
using namespace std;
#include <cuda.h>

struct Contact{
    char name[65];
    char phone_number[65];
};


string getInput(ifstream& file){
    string ans;
    char c;
    int readSuru = 0;
    while(file.get(c)){
        if(c == '\"'){
            if(readSuru == 1) break;
            readSuru = 1;
        }else{
            if(readSuru){
                ans.push_back(c);
            }
        }
    }
    return ans;
}

__device__ bool check(char* str1, char* str2){
    for(int i = 0; str1[i] != '\0'; i++){
        int flag = 1;
        for(int j = 0; str2[j] != '\0' ; j++){
            if(str1[i + j] != str2[j]){
                flag = 0;
                break;
            }
        }
        if(flag == 1) return true;
    }
    return false;
}


__global__ void myKernel(Contact* phoneBook, char* pat, int offset){
    int threadNumber = threadIdx.x + offset;
    if(check(phoneBook[threadNumber].name, pat)){
        printf("%s %s\n", phoneBook[threadNumber].name, phoneBook[threadNumber].phone_number);
    }
}



int main(int argc, char* argv[])
{
    int threadLimit = atoi(argv[2]);

    ifstream myfile("phonebook1.txt");
    vector<Contact> phoneBook;

    int count = 0;

    while(myfile.peek() != EOF){

        if(count > 10000) break;
        count++;

        string name = getInput(myfile);
        string phoneNum = getInput(myfile);

        Contact c;
        strcpy(c.name, name.c_str());
        strcpy(c.phone_number, phoneNum.c_str());

        phoneBook.push_back(c);
    }

    string search_name = argv[1];
    char pat[65];
    strcpy(pat, search_name.c_str());


    char* d_pat;
    cudaMalloc(&d_pat, 65); //memory allocation
    cudaMemcpy(d_pat, pat, 65, cudaMemcpyHostToDevice); //copying to device

    int n = phoneBook.size();
    Contact* d_phoneBook;
    cudaMalloc(&d_phoneBook, n*sizeof(Contact));
    cudaMemcpy(d_phoneBook, phoneBook.data(), n * sizeof(Contact), cudaMemcpyHostToDevice);


    int bakiAche = n;
    int offset = 0;
    while(bakiAche > 0){
        int batchSize = min(threadLimit, bakiAche);
        myKernel<<<1,batchSize>>>(d_phoneBook, d_pat, offset);
        cudaDeviceSynchronize();

        bakiAche -= batchSize;
        offset += batchSize;
    }

}

Overwriting phonebook_search.cu


In [52]:
!nvcc -arch=sm_75 phonebook_search.cu -o phonebook_search

In [53]:
!time ./phonebook_search SAD 10 > output.txt



real	0m0.270s
user	0m0.020s
sys	0m0.225s


In [54]:
!cat phonebook1.txt

"FATEMA JAHAN TAMMY","015 05 040"
"SADIA BINTA M RAHMAN","017 62 031"
"TAHSINA HAQUE NABILA","014 58 886"
"SAZNIN AKTER ZITU","016 16 217"
"ANTU RANI HOWLADAR","017 62 174"
"MASTURA ALAM","013 85 245"
"ASHAMONY","017 27 337"
"SAKIA RAHMAN","017 75 523"
"AISHWARYA ROY","014 51 584"
"FARIA BINTE MOHASIN","017 57 844"
"SUMAIYA AKTER SWEETY","018 07 741"
"SUNJIDA AKTER NIPA","012 20 350"
"MORIAM HASAN","014 07 824"
"ROUDALA SAHA","015 65 613"
"ASHIKA SIDDIKA","017 41 460"
"ASHA AKHTER ANNI","012 05 732"
"MST. NAZIA SULTANA MUMU","014 34 273"
"SUMAIA ISLAM","014 83 203"
"KATHA DEB","011 72 866"
"ISRAT JAHAN SHATHI","015 55 768"
"KHADIZATUL KOBRA","015 03 625"
"JERIN TASNIM","011 35 043"
"BIBI MORIOM TURABIN","015 40 555"
"NUSRAT JAHAN LAMIA","012 07 106"
"NUSRAT JAHAN MIM","013 66 223"
"NUSRAT JAHAN INTIA","013 25 233"
"AYESHA SIDDEKA TASMIAH","015 85 587"
"SANZIDA TASNIM","013 16 885"
"JARIN TASNIM JIDNE","014 13 616"
"TABASSUM TARIK","014 57 154"
"MARIYA AZAD DOLA","016 14 266"
"SADIA RAH

In [55]:
!cat output.txt

SADIA BINTA M RAHMAN 017 62 031
